<a href="https://colab.research.google.com/github/poddubnyoleg/nondualbot/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

http://karpathy.github.io/2015/05/21/rnn-effectiveness/

https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

https://stackoverflow.com/questions/48760472/how-to-use-the-keras-model-to-forecast-for-future-dates-or-events

https://github.com/danmoller/TestRepo/blob/master/TestBookLSTM.ipynb

In [1]:

import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, TimeDistributed
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import keras.backend as K
import sys

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
file = open('/content/drive/My Drive/nondualbot/i_am_that.txt', 'r') 
data = file.read() 

In [4]:
data[:100]

'That which permeates all, which nothing transcends and which, like the universal space around us, fi'

In [0]:
chars = sorted(list(set(data)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [6]:

n_chars = len(data)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  988168
Total Vocab:  55


For each chunk of data [seq_length] we take one-hot-encoded dataX array and dataY array (same but shifted +1)

In [43]:
seq_length = 100
batch_size = 100


K.clear_session()

model = Sequential()
model.add(LSTM(500, input_shape=(None, n_vocab), return_sequences=True, stateful=False))
model.add(Dropout(0.2))
model.add(LSTM(250, input_shape=(None, n_vocab), return_sequences=True, stateful=False))
model.add(Dropout(0.2))
model.add(LSTM(250, input_shape=(None, n_vocab), return_sequences=True, stateful=False))
model.add(Dropout(0.2))

model.add(Dense(n_vocab, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, None, 500)         1112000   
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 500)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 250)         751000    
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 250)         0         
_________________________________________________________________
lstm_3 (LSTM)                (None, None, 250)         501000    
_________________________________________________________________
dropout_3 (Dropout)          (None, None, 250)         0         
_________________________________________________________________
dense_1 (Dense)              (None, None, 55)          13805     
Total para

In [0]:
filename = "/content/drive/My Drive/nondualbot/best_current_weights_2.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
filepath="/content/drive/My Drive/nondualbot/best_current_weights_2.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [0]:
num_data = [char_to_int[char] for char in data]

**train with batch generator with fixed or variable batch sequense size**
* Inner state doesnt share between samples inside of one batch, but can share between batches, if stateful=True
* fit(shuffle=) doesn't relate to RNNs, because to keep inner state consistent withi epoch, you need to set stateful=True and batch_size=1, so there is nothing to shuffle
* https://stackoverflow.com/questions/43882796/when-does-keras-reset-an-lstm-state




In [0]:
batch_size = 100

def train_generator():
  while True:

    #seq_length = np.random.randint(low=2, high=200)
    #x_train, y_train = [], []
    
    cur_pos = np.random.randint(low=0, high=n_chars-batch_size-1)

    x_train = np_utils.to_categorical(num_data[cur_pos:cur_pos+batch_size], num_classes=n_vocab)
    y_train = np_utils.to_categorical(num_data[cur_pos+1:cur_pos+batch_size+1], num_classes=n_vocab)

    yield np.reshape(x_train, (1, batch_size, n_vocab)), np.reshape(y_train, (1, batch_size, n_vocab))


In [0]:
model.fit_generator(train_generator(), steps_per_epoch=100, epochs=100, verbose=1, callbacks=callbacks_list)


Epoch 1/100
100/100 [==============================] - 51s 513ms/step - loss: 3.1208

Epoch 00001: loss improved from inf to 3.12076, saving model to /content/drive/My Drive/nondualbot/best_current_weights_2.hdf5
Epoch 2/100
100/100 [==============================] - 49s 493ms/step - loss: 3.0235

Epoch 00002: loss improved from 3.12076 to 3.02352, saving model to /content/drive/My Drive/nondualbot/best_current_weights_2.hdf5
Epoch 3/100
100/100 [==============================] - 49s 494ms/step - loss: 2.9900

Epoch 00003: loss improved from 3.02352 to 2.98999, saving model to /content/drive/My Drive/nondualbot/best_current_weights_2.hdf5
Epoch 4/100
100/100 [==============================] - 49s 492ms/step - loss: 2.7599

Epoch 00004: loss improved from 2.98999 to 2.75994, saving model to /content/drive/My Drive/nondualbot/best_current_weights_2.hdf5
Epoch 5/100
100/100 [==============================] - 49s 491ms/step - loss: 2.6700

Epoch 00005: loss improved from 2.75994 to 2.67005

## Create new stateful model with same weights and passing cell state for next prediction
No need to pass all the data from beginning, can feed with 1 example per model.predict, saving state from previous prediction
https://stackoverflow.com/questions/48760472/how-to-use-the-keras-model-to-forecast-for-future-dates-or-events

In [21]:

model2 = Sequential()
model2.add(LSTM(250, batch_input_shape=(1, None, n_vocab), return_sequences=True, stateful=True))
model2.add(Dropout(0.2))
model2.add(LSTM(125, batch_input_shape=(1, None, n_vocab), return_sequences=False, stateful=True))
model2.add(Dropout(0.2))

model2.add(Dense(n_vocab, activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam')
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (1, None, 250)            306000    
_________________________________________________________________
dropout_3 (Dropout)          (1, None, 250)            0         
_________________________________________________________________
lstm_4 (LSTM)                (1, 125)                  188000    
_________________________________________________________________
dropout_4 (Dropout)          (1, 125)                  0         
_________________________________________________________________
dense_2 (Dense)              (1, 55)                   6930      
Total params: 500,930
Trainable params: 500,930
Non-trainable params: 0
_________________________________________________________________


In [0]:
filename = "/content/drive/My Drive/nondualbot/best_current_weights_2.hdf5"
model2.load_weights(filename)
model2.compile(loss='categorical_crossentropy', optimizer='adam')

Predict next letter, save state, predict next letter with saved state

In [40]:
model2.reset_states()

question = 'That which permeates all.'
question_features = np_utils.to_categorical([char_to_int[char] for char in question], num_classes=n_vocab)
#no need to fill array with zeros
#question_features = np.append(np.zeros(shape=(max(seq_length-question_features.shape[0],0), n_vocab)), question_features, axis=0)

print(question)

#predict question char by char

sys.stdout.write(' ')

for question_char in question_features:
  model_response = model2.predict(question_char.reshape(1,1,n_vocab))
  char_n = np.argmax(model_response)
  print_char = int_to_char[char_n]
  sys.stdout.write(print_char)

sys.stdout.write('\n')

for _ in range(150):
  model_input = np.zeros(n_vocab)
  model_input[char_n] = 1
  model_response = model2.predict(model_input.reshape(1,1,n_vocab))[0]
  char_n = np.random.choice(len(model_response), p=(model_response**2)/sum(model_response**2))#np.argmax(model_response)#
  print_char = int_to_char[char_n]
  sys.stdout.write(print_char)

That which permeates all.
 het ihach iersenne  anl  
I am is self and mate is the present to the body is the porson is the knower is the juttent is with will undinting and the mind is a with and a cannot

#Old prediction

In [27]:
question = 'That which permeates all.'
question_features = np_utils.to_categorical([char_to_int[char] for char in question], num_classes=n_vocab)
question_features = np.append(np.zeros(shape=(max(seq_length-question_features.shape[0],0), n_vocab)), question_features, axis=0)
print(question)

test_prediction = model.predict(question_features.reshape(1,len(question_features),n_vocab))[0]
sys.stdout.write(' ')
for test_char in test_prediction:
  print_char = int_to_char[np.argmax(test_char)]
  sys.stdout.write(print_char)

That which permeates all.
   eeis                                                                      eniahach iersenne  anl  

In [31]:
question = 'That which permeates all.'
pattern = np_utils.to_categorical([char_to_int[char] for char in question], num_classes=n_vocab)
pattern = np.append(np.zeros(shape=(max(seq_length-pattern.shape[0],0), n_vocab)), pattern, axis=0)

print(question)

test_prediction = model.predict(pattern.reshape(1,len(pattern),n_vocab))[0]
sys.stdout.write(' ')
for test_char in test_prediction:
  print_char = int_to_char[np.argmax(test_char)]
  sys.stdout.write(print_char)

sys.stdout.write('\n')
# generate characters
for i in range(150):

  prediction = model.predict(np.reshape(pattern, (1, pattern.shape[0], pattern.shape[1])), verbose=0)[0][-1]
  
  index = np.argmax(prediction)#np.random.choice(len(prediction), p=(prediction**2)/sum(prediction**2))#
  result = int_to_char[index]
  sys.stdout.write(result)
  pattern = np.append(pattern, [np_utils.to_categorical(index,num_classes=n_vocab)], axis=0)
  pattern = pattern[1:len(pattern)]

That which permeates all.
   eeis                                                                      eniahach iersenne  anl  
 I am the world and the see the world and the see the world and the see the world and the see the world and the see the world and the see the world an

In [24]:
model.predict(np.reshape(pattern, (1, pattern.shape[0], pattern.shape[1])), verbose=0)[0][-1]

array([9.9999940e-01, 5.4691748e-07, 4.8891195e-12, 1.0469446e-09,
       2.6797808e-10, 6.7335869e-22, 2.8095636e-26, 6.3552258e-29,
       7.0487436e-23, 6.3646487e-26, 3.1674805e-26, 4.9144446e-28,
       1.0800542e-21, 1.1273043e-25, 3.6994567e-28, 6.7637692e-34,
       3.3639891e-28, 2.0014950e-27, 2.9739306e-31, 5.9701945e-25,
       2.0236516e-27, 0.0000000e+00, 1.8087869e-34, 1.6505187e-24,
       1.3041807e-22, 2.8014332e-31, 8.8703745e-35, 1.8485967e-19,
       6.2777927e-26, 6.5178172e-19, 1.2295411e-16, 1.3757429e-19,
       6.6819938e-13, 3.0502608e-13, 2.4411573e-12, 2.3483241e-15,
       1.7099024e-17, 1.6124142e-16, 2.0943558e-22, 2.6491773e-17,
       1.4324105e-12, 8.1101988e-14, 7.6933120e-11, 5.4370323e-14,
       4.9228969e-17, 4.9811773e-23, 2.1248222e-15, 8.4897835e-12,
       2.5380966e-16, 1.9712886e-15, 8.8841031e-17, 3.3209028e-15,
       2.6880777e-22, 6.2216196e-20, 2.2106736e-28], dtype=float32)

In [58]:
np.append(pattern, [np_utils.to_categorical(index,num_classes=n_vocab)], axis=0)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [57]:
patte

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.], dtype=float32)

In [23]:
np.argmax(dataY[0])

40

In [24]:
dataY[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.], dtype=float32)